This task intends to measure your deep learning basic skills. We will provide you a dataset containing 131 different fruits. Download it from this [link](https:///www.kaggle.com/moltean/fruits), download it and upload it to your copy of this file

You have to implement a Convolutional Neural Network to classify the input fruit image. For this task, you must follow the following rules:

- Use a ResNet50 as your CNN backbone.
- The model output must be a probability score for all the classes.
- You can implement the model in any Deep Learning framework that you are used to use.
- Make sure you write your code following the best practices.
- Make sure you document all the steps you took to solve the problem.


Importing kaggle's dataset

In [1]:
! pip install -q kaggle
#from google.colab import files
#files.upload()

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory '/home/codemaster/.kaggle': File exists


In [3]:
!kaggle datasets download -d moltean/fruits


fruits.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!mkdir data
!unzip fruits.zip -d data

mkdir: cannot create directory 'data': File exists
Archive:  fruits.zip
replace data/fruits-360-original-size/fruits-360-original-size/Meta/apple_6/info.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


Importing libs

In [15]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tensorflow.keras.models import Sequential


from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
import tensorflow as tf

In [16]:
# dimensions of our images.
img_width, img_height = 224, 224 # we set the img_width and img_height according to the pretrained models we are
# going to use. The input size for ResNet-50 is 224 by 224 by 3.

train_data_dir = 'data/fruits-360_dataset/fruits-360/Training'
test_data_dir = 'data/fruits-360_dataset/fruits-360/Test/'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size = 16

Making generators for the data

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [18]:
!pip install plotly==5.3.1

In [19]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [20]:
training_data = pd.DataFrame(train_generator.classes, columns=['classes'])
validation_data = pd.DataFrame(validation_generator.classes, columns=['classes'])

In [21]:
def create_stack_bar_data(col, df):
    aggregated = df[col].value_counts().sort_index()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

In [22]:
x1, y1 = create_stack_bar_data('classes', training_data)
x1 = list(train_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="Class Count")
layout = dict(height=400, width=1200, title='Class Distribution in Training Data', legend=dict(orientation="h"), 
                yaxis = dict(title = 'Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
fig.show(renderer="colab")

In [23]:
x1, y1 = create_stack_bar_data('classes', validation_data)
x1 = list(validation_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="Class Count")
layout = dict(height=400, width=1200, title='Class Distribution in validation Data', legend=dict(orientation="h"), 
                yaxis = dict(title = 'Class Count'))
fig = go.Figure(data=[trace1], layout=layout)
fig.show(renderer="colab")

With the data distribution, we can see that we have classes with much more data than the others, for simplicity we will ignore it because we can get a good accuracy with the dataset just the way the things are set. 

build the models:
we gonna implement two ResNet-50, pre-trained model with ImageNet and a basic implementation without the weights.

Here, we load the ResNet-50 with the ImageNet weights. We remove the top so that we can add our own layer according to the number of our classes. We then add our own layers to complete the model architecture.

In [17]:
#import inception with pre-trained weights. do not include fully #connected layers
inception_base = applications.ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = inception_base.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(131, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

94781440/94765736 [==============================] - 24s 0us/step


Vanilla implementation

In [18]:
#import inception with pre-trained weights. do not include fully #connected layers
inception_base_vanilla = applications.ResNet50(weights=None, include_top=False)

# add a global spatial average pooling layer
x = inception_base_vanilla.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a fully connected output/classification layer
predictions = Dense(131, activation='softmax')(x)
# create the full network so we can train on it
inception_transfer_vanilla = Model(inputs=inception_base_vanilla.input, outputs=predictions)

Compiling the Models:
We set the loss function, the optimization algorithm to be used and metrics to be calculated at the end of each epoch.

In [19]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])

inception_transfer_vanilla.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])

Note:In this computer i don't have the gpu to make the training and analyse the results, from this point i can't run the cells because the training for this model is heavy.

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16631753966683133150
]


[]

In [ ]:
with tf.device("/device:GPU:0"):
    history_vanilla = inception_transfer_vanilla.fit_generator(
    train_generator,
    epochs=5, shuffle = True, verbose = 1, validation_data = validation_generator)


In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_vanilla.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_vanilla.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()